In [ ]:
#export
import os
import uuid
from fastai2.basics import *
from fastai2.learner import Callback

In [ ]:
from nbdev.showdoc import *

In [ ]:
#default_exp callback.neptune

# Neptune.ai

> Integration with [neptune.ai](https://www.neptune.ai)

First thing first.

1. You need to install neptune-client. In your terminal run:

```
pip install neptune-client
```

or (alternative installation using conda). In your terminal run:

```
conda install neptune-client -c conda-forge
```

2. Create **free** account: [neptune.ai/register](https://neptune.ai/register).
3. Export API token to the environment variable (more help [here](https://docs.neptune.ai/python-api/tutorials/get-started.html#copy-api-token)). In your terminal run:

```
export NEPTUNE_API_TOKEN='YOUR_LONG_API_TOKEN'
```

or append the command above to your `~/.bashrc` or `~/.bash_profile` files (**recommended**). More help is [here](https://docs.neptune.ai/python-api/tutorials/get-started.html#copy-api-token).

## How to use?
Key is to call `neptune.init()` before you create `Learner()` and call `neptune_create_experiment()`, before you fit the model.

Use `NeptuneCallback` in your `Learner`, like this:

```
from fastai2.callback.neptune import NeptuneCallback

neptune.init('USERNAME/PROJECT_NAME')  # specify project

learn = Learner(dls, model,
                cbs=NeptuneCallback()
                )

neptune.create_experiment()  # start experiment
learn.fit_one_cycle(1)
```

In [ ]:
#export
import neptune

In [ ]:
#export
class NeptuneCallback(Callback):
    "Log losses, metrics, model weights"
    def __init__(self, save_model=True, log_preds=True, n_preds=15):
        self.neptune_save_model = save_model
        self.log_preds = log_preds
        self.n_preds = n_preds
        self.model_summary_name = 'model_summary_{}.txt'.format(str(uuid.uuid1())[:8])
        self.neptune_exp = None

        if neptune.project is None:
            raise ValueError('You did not initialize project in neptune.\n',
                             'Please invoke `neptune.init("USERNAME/PROJECT_NAME")` before this callback.')

    def begin_fit(self):
        try:
            self.neptune_exp = neptune.get_experiment()
        except ValueError:
            print('No active experiment. Please invoke `neptune.create_experiment()` before this callback.')

        try:
            self.neptune_exp.set_property('n_epoch', str(self.learn.n_epoch))
            self.neptune_exp.set_property('model_class', str(type(self.learn.model)))
        except:
            print('Did not log all properties. Check properties in the {}.'.format(neptune.get_experiment()))

        try:
            with open(self.model_summary_name, 'w') as f:
                f.write(repr(self.learn.model))
            self.neptune_exp.log_artifact(self.model_summary_name, 'model_summary.txt')
            os.remove(self.model_summary_name)
        except:
            print('Did not log model summary. Check if your model is PyTorch model.')

        if self.neptune_save_model and not hasattr(self.learn, 'save_model'):
            print('Unable to log model to Neptune.\n',
                  'Use "SaveModelCallback" to save model checkpoints that will be logged to Neptune.')

    def after_batch(self):
        self.neptune_exp.set_property('n_iter', str(self.learn.n_iter))
        if self.learn.training:
            self.neptune_exp.log_metric('batch__smooth_loss', self.learn.smooth_loss)
            self.neptune_exp.log_metric('batch__train_iter', self.learn.train_iter)
            for i, h in enumerate(self.learn.opt.hypers):
                for k, v in h.items():
                    self.neptune_exp.log_metric('batch__opt.hypers.{}'.format(k), v)

    def after_epoch(self):
        # log metrics
        for n, v in zip(self.learn.recorder.metric_names, self.learn.recorder.log):
            if n not in ['epoch', 'time']:
                self.neptune_exp.log_metric('epoch__{}'.format(n), v)
            if n == 'time':
                self.neptune_exp.log_text('epoch__{}'.format(n), str(v))

        # log model weights
        if self.neptune_save_model and hasattr(self.learn, 'save_model'):
            if self.learn.save_model.every_epoch:
                _file = join_path_file('{}_{}'.format(self.learn.save_model.fname, self.learn.save_model.epoch),
                                       self.learn.path / self.learn.model_dir,
                                       ext='.pth')
            else:
                _file = join_path_file(self.learn.save_model.fname,
                                       self.learn.path / self.learn.model_dir,
                                       ext='.pth')
            self.neptune_exp.log_artifact(_file)

    def after_fit(self):
        try:
            self.neptune_exp.stop()
        except:
            print('No neptune experiment to stop.')
